In [3]:
import os
import numpy as np

from pyuvdata import UVData
from pyuvdata.data import DATA_PATH

In [4]:
phasedfile = os.path.join(DATA_PATH, 'mwa_corr_fits_testfiles/1131733552_mini_cotter_phased.uvfits')
# this file has phased the uvw array but not the data array
# unphasedfile = os.path.join(DATA_PATH, 'mwa_corr_fits_testfiles/1131733552_mini_cotter.uvfits')
rawfile = os.path.join(DATA_PATH, 'mwa_corr_fits_testfiles/1131733552_20151116182537_mini_gpubox01_00.fits')
metafile = os.path.join(DATA_PATH, 'mwa_corr_fits_testfiles/1131733552.metafits')
cotter_phased = UVData()
# cotter_unphased = UVData()
mwa_raw = UVData()
cotter_phased.read(phasedfile)
mwa_phased = UVData()
# cotter_unphased.read(unphasedfile)
# get raw data to phase with gcrs frame
mwa_raw.read_mwa_corr_fits([metafile, rawfile], correct_cable_len=True)
# this phases the raw data with icrs frame
mwa_phased.read_mwa_corr_fits([metafile, rawfile], correct_cable_len=True, phase_data=True)

telescope_location is not set. Using known values for MWA.
some coarse channel files were not submitted


In [5]:
ra = cotter_phased.phase_center_ra
dec = cotter_phased.phase_center_dec
mwa_uv_phased1=mwa_raw.copy()
mwa_uv_phased2=mwa_raw.copy()
mwa_uv_phased1.phase(ra, dec, phase_frame='gcrs')
mwa_uv_phased2.phase(ra, dec, phase_frame='gcrs', use_ant_pos=True)

In [6]:
# cotter is missing xy autos; yx autos are conjugated
autos = np.isclose(cotter_phased.ant_1_array - cotter_phased.ant_2_array, 0.0)
cotter_phased.data_array[autos, :, :, 2] = cotter_phased.data_array[autos, :, :, 3]
cotter_phased.data_array[autos, :, :, 3] = np.conj(cotter_phased.data_array[autos, :, :, 3])

In [14]:
# compare phased raw data with cotter phased data
print('maximum difference in uvw_array in meters')
print(np.max(np.abs(cotter_phased.uvw_array-mwa_uv_phased1.uvw_array)))
abs_diff = np.abs(cotter_phased.data_array-mwa_uv_phased1.data_array)
abs_sum = np.abs(cotter_phased.data_array+mwa_uv_phased1.data_array)
print('maximum ratio of 2 * data_array difference / data_array sum')
print(np.max(2*abs_diff/abs_sum))

maximum difference in uvw_array in meters
0.0073829564281595594
maximum ratio of 2 * data_array difference / data_array sum
0.030406464


In [8]:
# compare phased raw data, use_ant_pos-True, with cotter phased data
print('maximum difference in uvw_array in meters')
print(np.max(np.abs(cotter_phased.uvw_array-mwa_uv_phased2.uvw_array)))
abs_diff = np.abs(cotter_phased.data_array-mwa_uv_phased2.data_array)
abs_sum = np.abs(cotter_phased.data_array+mwa_uv_phased2.data_array)
print('maximum ratio of 2 * data_array difference / data_array sum')
print(np.max(2*abs_diff/abs_sum))

maximum difference in uvw_array in meters
0.007378927604122509
maximum ratio of 2 * data_array difference / data_array sum
0.03038985


In [9]:
# compare icrs phased data to cotter
print('maximum difference in uvw_array in meters')
print(np.max(np.abs(cotter_phased.uvw_array-mwa_phased.uvw_array)))
abs_diff = np.abs(cotter_phased.data_array-mwa_phased.data_array)
abs_sum = np.abs(cotter_phased.data_array+mwa_phased.data_array)
print('maximum ratio of 2 * data_array difference / data_array sum')
print(np.max(2*abs_diff/abs_sum))

maximum difference in uvw_array in meters
0.1442541375743076
maximum ratio of 2 * data_array difference / data_array sum
0.029768882


In [10]:
# nsample_arrays won't match; make sure they are consistent
cotter_phased.nsample_array[np.where(mwa_uv_phased1.nsample_array == 1.0)] = 1.0

In [11]:
# adjust other parameters that don't match
cotter_phased.flag_array = mwa_uv_phased1.flag_array
cotter_phased.history = mwa_uv_phased1.history
cotter_phased.vis_units = mwa_uv_phased1.vis_units
cotter_phased.extra_keywords = mwa_uv_phased1.extra_keywords
cotter_phased.phase_center_frame = mwa_uv_phased1.phase_center_frame
mwa_uv_phased1.dut1 = cotter_phased.dut1
mwa_uv_phased1.earth_omega = cotter_phased.earth_omega
mwa_uv_phased1.gst0 = cotter_phased.gst0
mwa_uv_phased1.rdate = cotter_phased.rdate
mwa_uv_phased1.timesys = cotter_phased.timesys

In [12]:
# change tols so data_array and uvw_array match
# tols are (relative, absolute)
cotter_phased._data_array.tols = (0, 13)
mwa_uv_phased1._data_array.tols = (0, 13)
# if atol = 0 and array contains zeros, np.isclose returns false
cotter_phased._uvw_array.tols = (0, .008)
mwa_uv_phased1._uvw_array.tols = (0, .008)
assert cotter_phased == mwa_uv_phased1
